# **Bài thực hành: Biểu diễn Chuỗi cho Mạng Nơ-ron Hồi quy (RNNs)**

* **Họ và tên:** Huỳnh Thế Hy
* **Mã số sinh viên:** 051205009083

## **1. Các Kỹ thuật Mã hóa Số học**

Chuyển đổi các phần tử (token) trong chuỗi thành các giá trị số.

### **Mã hóa số nguyên (Integer Encoding)**

Gán một số nguyên duy nhất cho mỗi token riêng biệt trong từ điển.

**Giải thích:**
- **Xây dựng từ điển:** Duyệt qua toàn bộ tập dữ liệu để tìm ra tất cả các token duy nhất (ví dụ: các từ).
- **Gán ID:** Tạo hai cấu trúc dữ liệu: một map từ token sang số nguyên (ID) và một map ngược lại từ ID sang token.

In [1]:
# Dữ liệu văn bản mẫu bằng tiếng Anh
corpus = [
    "deep learning is an exciting field",
    "deep learning is developing very fast",
    "artificial intelligence is the future"
]

# Tách các câu thành từng từ
words = set()
for sentence in corpus:
    for word in sentence.split(' '):
        words.add(word)

# Sắp xếp để đảm bảo thứ tự nhất quán
words = sorted(list(words))

# Xây dựng từ điển: map từ sang số nguyên và ngược lại
word_to_int = {word: i+1 for i, word in enumerate(words)} # Bắt đầu từ 1, 0 dành cho padding
int_to_word = {i+1: word for i, word in enumerate(words)}

print("--- Vocabulary ---")
print(word_to_int)
print("\n--- Original Sentence ---")
print(corpus[0])

# Mã hóa câu đầu tiên thành chuỗi số nguyên
integer_encoded = [word_to_int[word] for word in corpus[0].split(' ')]
print("\n--- Câu đã được mã hóa số nguyên ---")
print(integer_encoded)

--- Vocabulary ---
{'an': 1, 'artificial': 2, 'deep': 3, 'developing': 4, 'exciting': 5, 'fast': 6, 'field': 7, 'future': 8, 'intelligence': 9, 'is': 10, 'learning': 11, 'the': 12, 'very': 13}

--- Original Sentence ---
deep learning is an exciting field

--- Câu đã được mã hóa số nguyên ---
[3, 11, 10, 1, 5, 7]


### **One-Hot Encoding**

Biểu diễn mỗi token bằng một vector nhị phân. Vector này có độ dài bằng kích thước của từ điển, chứa toàn số 0 ngoại trừ vị trí tương ứng với chỉ số của token đó sẽ là 1.

**Giải thích:**
- **Ưu điểm:** Biểu diễn rõ ràng, không tạo ra mối quan hệ thứ tự giả tạo giữa các từ.
- **Nhược điểm:** Tạo ra các vector rất lớn và thưa thớt (sparse) khi từ điển có kích thước lớn, dẫn đến tốn kém bộ nhớ và tính toán.


In [2]:
import torch
import torch.nn.functional as F

# Lấy kích thước từ điển
vocab_size = len(word_to_int) + 1 # +1 vì chúng ta bắt đầu từ 1

# Câu đã được mã hóa số nguyên từ ví dụ trước
integer_encoded_tensor = torch.tensor(integer_encoded)

print(f"Kích thước từ điển: {vocab_size}")
print(f"Chuỗi số nguyên: {integer_encoded_tensor}")

# Sử dụng one_hot của PyTorch
one_hot_encoded = F.one_hot(integer_encoded_tensor, num_classes=vocab_size)

print("\n--- Vector One-Hot cho câu đầu tiên ---")
print(f"Kích thước của tensor đầu ra: {one_hot_encoded.shape}") # (số từ, kích thước từ điển)
print("Vector One-Hot (chỉ hiển thị 3 từ đầu tiên):")
print(one_hot_encoded[:3])

Kích thước từ điển: 14
Chuỗi số nguyên: tensor([ 3, 11, 10,  1,  5,  7])

--- Vector One-Hot cho câu đầu tiên ---
Kích thước của tensor đầu ra: torch.Size([6, 14])
Vector One-Hot (chỉ hiển thị 3 từ đầu tiên):
tensor([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])


### **Embeddings**

Với dữ liệu có số lượng lớn các hạng mục như từ vựng trong ngôn ngữ. Embeddings học cách biểu diễn mỗi token bằng một vector số thực, dày đặc (dense) và có số chiều thấp.

**Giải thích:**
- **Ưu điểm:**
    - **Tiết kiệm không gian:** Vector có số chiều nhỏ hơn nhiều so với one-hot encoding.
    - **Nắm bắt ngữ nghĩa:** Các từ có ngữ nghĩa tương tự nhau sẽ có vector biểu diễn gần nhau trong không gian vector. Mối quan hệ này được mô hình tự học trong quá trình huấn luyện.
- **Cách hoạt động trong PyTorch:** Lớp `torch.nn.Embedding` hoạt động như một bảng tra cứu (lookup table). Nó lưu trữ các vector embedding cho toàn bộ từ điển. Khi nhận đầu vào là một chuỗi các chỉ số nguyên, nó sẽ trả về chuỗi các vector embedding tương ứng.


In [3]:
import torch.nn as nn

# Các tham số
vocab_size = len(word_to_int) + 1  # Kích thước từ điển
embedding_dim = 10                 # Số chiều của vector embedding (tùy chọn)

# Khởi tạo lớp Embedding
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# Chuỗi số nguyên đầu vào
input_tensor = torch.tensor(integer_encoded, dtype=torch.long)

# Lấy vector embedding
word_embeddings = embedding_layer(input_tensor)

print(f"--- Kích thước đầu vào (chuỗi số nguyên): {input_tensor.shape} ---")
print(input_tensor)
print(f"\n--- Kích thước đầu ra (chuỗi vector embedding): {word_embeddings.shape} ---")
print("(Mỗi từ giờ được biểu diễn bằng một vector 10 chiều)")
print(word_embeddings.detach().numpy()) # detach() để không tính gradient

--- Kích thước đầu vào (chuỗi số nguyên): torch.Size([6]) ---
tensor([ 3, 11, 10,  1,  5,  7])

--- Kích thước đầu ra (chuỗi vector embedding): torch.Size([6, 10]) ---
(Mỗi từ giờ được biểu diễn bằng một vector 10 chiều)
[[ 0.68502814 -0.87615246 -1.3241562  -0.3501906  -0.9227804  -0.76528347
  -1.5597167   0.21562248 -0.43947715  0.31613848]
 [ 0.7970177   0.08856621 -0.33122867 -2.3981686   0.61012113 -1.1176087
   1.1684937  -0.63378465  0.22844046 -1.24754   ]
 [-0.15496658 -0.46815896  0.37941292 -1.0699453   0.06545174 -1.8102114
   0.4133599  -0.73862785 -0.05509819  0.53472275]
 [ 0.47963604 -0.900339   -2.0970433  -0.24928004  0.3107327  -0.5752461
   1.3505844  -0.08419859  1.3346554  -1.0016862 ]
 [ 0.8626436   1.50018    -0.80384594  0.52670205  0.17610516 -1.0592409
   1.070216    0.07873873  0.17420557 -0.63329965]
 [ 0.02724205  0.65253603  1.8971881   0.42211938 -0.2570622   1.5119644
   0.21019076 -1.5072191   1.037056    0.7171995 ]]


## **2. Xử lý Chuỗi có Độ dài Thay đổi (Handling Variable Lengths)**

Hai kỹ thuật chính: **Padding** và **Truncation**.

### **Padding**

Padding là kỹ thuật thêm các token đặc biệt (thường là số 0) vào cuối các chuỗi ngắn hơn để làm cho chúng có độ dài bằng với chuỗi dài nhất trong lô.

In [4]:
# Mã hóa toàn bộ corpus
sequences = [[word_to_int[word] for word in s.split()] for s in corpus]
print("--- Các chuỗi trước khi padding ---")
for seq in sequences:
    print(f"Độ dài={len(seq)}: {seq}")

# Tìm độ dài của chuỗi dài nhất
max_length = max(len(seq) for seq in sequences)

# Thực hiện padding
padded_sequences = []
for seq in sequences:
    # Lấy độ dài cần pad
    pad_length = max_length - len(seq)
    # Tạo tensor chứa các số 0 và nối vào cuối chuỗi
    padded_seq = seq + [0] * pad_length
    padded_sequences.append(padded_seq)

print(f"\n--- Các chuỗi sau khi padding (đến độ dài {max_length}) ---")
for seq in padded_sequences:
    print(f"Độ dài={len(seq)}: {seq}")

# Chuyển thành tensor
padded_tensor = torch.tensor(padded_sequences)
print("\n--- Tensor cuối cùng ---")
print(padded_tensor)

--- Các chuỗi trước khi padding ---
Độ dài=6: [3, 11, 10, 1, 5, 7]
Độ dài=6: [3, 11, 10, 4, 13, 6]
Độ dài=5: [2, 9, 10, 12, 8]

--- Các chuỗi sau khi padding (đến độ dài 6) ---
Độ dài=6: [3, 11, 10, 1, 5, 7]
Độ dài=6: [3, 11, 10, 4, 13, 6]
Độ dài=6: [2, 9, 10, 12, 8, 0]

--- Tensor cuối cùng ---
tensor([[ 3, 11, 10,  1,  5,  7],
        [ 3, 11, 10,  4, 13,  6],
        [ 2,  9, 10, 12,  8,  0]])


### **Truncation**

Truncation là kỹ thuật cắt bớt các chuỗi dài hơn một ngưỡng (maximum length) cho trước để đảm bảo tính nhất quán và giảm tải tính toán.

In [5]:
max_len_truncate = 5

truncated_sequences = []
for seq in sequences:
    # Cắt chuỗi nếu nó dài hơn max_len_truncate
    truncated_seq = seq[:max_len_truncate]
    truncated_sequences.append(truncated_seq)

print(f"--- Các chuỗi sau khi truncation (độ dài tối đa {max_len_truncate}) ---")
for seq in truncated_sequences:
    print(f"Độ dài={len(seq)}: {seq}")

# Kết hợp cả Truncation và Padding
final_sequences = []
final_length = 5
for seq in sequences:
    # 1. Cắt bớt
    truncated = seq[:final_length]
    # 2. Đệm
    pad_len = final_length - len(truncated)
    padded = truncated + [0] * pad_len
    final_sequences.append(padded)
    
print("\n--- Các chuỗi sau khi kết hợp Truncation và Padding ---")
for seq in final_sequences:
    print(f"Độ dài={len(seq)}: {seq}")

final_tensor = torch.tensor(final_sequences)
print("\n--- Tensor cuối cùng sẵn sàng cho mô hình ---")
print(final_tensor)

--- Các chuỗi sau khi truncation (độ dài tối đa 5) ---
Độ dài=5: [3, 11, 10, 1, 5]
Độ dài=5: [3, 11, 10, 4, 13]
Độ dài=5: [2, 9, 10, 12, 8]

--- Các chuỗi sau khi kết hợp Truncation và Padding ---
Độ dài=5: [3, 11, 10, 1, 5]
Độ dài=5: [3, 11, 10, 4, 13]
Độ dài=5: [2, 9, 10, 12, 8]

--- Tensor cuối cùng sẵn sàng cho mô hình ---
tensor([[ 3, 11, 10,  1,  5],
        [ 3, 11, 10,  4, 13],
        [ 2,  9, 10, 12,  8]])


## **Kết luận**

Các kỹ thuật cơ bản nhưng vô cùng thiết yếu để chuẩn bị dữ liệu chuỗi cho mô hình RNN. Việc lựa chọn phương pháp mã hóa (Integer, One-Hot, hay Embedding) và cách xử lý độ dài chuỗi (Padding, Truncation) phụ thuộc vào bài toán cụ thể và nguồn tài nguyên tính toán.